In [1]:
import os, numpy, PIL
from PIL import Image
import datetime
import time
import cv2
import os

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [64]:
def create_video_panning (ip_dir, op_dir, op_resolution, frame_rate=24, ext='.jpg', \
                          panning_style='h', aspect_ratio=(2, 3), vid_ext='.avi'):
    
    # Get the list of images
    images = [img for img in os.listdir(ip_dir) if img.endswith(ext)]
    images.sort()
    
    total_frames = len(images)
    
    if (total_frames == 0):
        print('No images found with extension ' + ext)
        return
    
    # Get image parameters such as width and height
    sample_image = os.path.join(ip_dir, images[0])
    img_h, img_w = get_image_parameters(sample_image) # Assumes all images are of same resolution
    
    op_width, op_height = op_resolution
    # Check the output_resolution
    if (op_height <= 0):
        op_height = img_h
    if (op_width <= 0):
        op_width = img_w
        
    op_resolution = (op_width, op_height)
        
    pixels_to_move_per_frame = get_horizontal_panning_parameters(img_w, total_frames, op_resolution)
    
    print(img_h, img_w, pixels_to_move_per_frame, total_frames, op_resolution)
    
    #Aspect ratio
    aspect_w, aspect_h = aspect_ratio
    
    
    # Check the desired width and height of the output image for filename
    width, height = op_resolution
    splt = ip_dir.split('/')
    sub_dir = splt[len(splt)-1]
    op_file = os.path.join(op_dir, sub_dir+'_'+str(width)+'x'+str(height)+vid_ext)
    
    print('Output Video: ', op_file, '\n')

    # Create a video object for the final output file
    codec = 'mp4v'
    fourcc = cv2.VideoWriter_fourcc(*codec)
    video = cv2.VideoWriter(op_file, fourcc, frame_rate, op_resolution, True)

    temp_file = 'temp.jpg'

    frame_count = 0
    for image in images:

        frame_count += 1
        
        image = os.path.join(ip_dir, image)

        
        if (panning_style == 'h'):
            x_start, x_width, y_start, y_height = get_horizontal_image_panning_parameters( \
                    img_h, img_w, frame_count, total_frames, op_resolution, pixels_to_move_per_frame)
#        print('Op resolution after for ', op_resolution)
        
#        print('image dimensions', x_start, x_width, y_start, y_height)
        im = crop_image(image, x_start, x_width, y_start, y_height)
        
        if ((aspect_w>0) and (aspect_h>0)):
#            print(aspect_ratio, x_width, y_height, panning_style)
            op_resolution = get_dimensions_aspect_ratio(aspect_ratio, x_width, y_height, panning_style)
        
        print(op_resolution, end='\r')
        # im = Image.open(image)
        im_resized = im.resize(op_resolution, Image.ANTIALIAS)
        im_resized.save(temp_file)

        video.write(cv2.imread(temp_file))
        
        print('Frames ('+str(frame_count)+'/'+str(total_frames)+')', end='\r')

    os.remove(temp_file)

    cv2.destroyAllWindows()
    video.release()

In [63]:
def get_dimensions_aspect_ratio(aspect_ratio, x_width, y_height, panning_style):
    aspect_w, aspect_h = aspect_ratio
    
    # If it is a horizontal panning, the width is not changed
    if (panning_style == 'h'):
        # Check if aspect ratio of x_width:y_height is greater than w:h, change the y_height
        if (aspect_w < aspect_h):
            if ((float(x_width)/y_height) < (float(aspect_w)/aspect_h)):
                y_height = int(round(float(aspect_h)/aspect_w * x_width))
            
    return x_width, y_height

In [21]:
t_w, t_h = get_dimensions_aspect_ratio((2,3), 1920, 4000, 'h')

print(t_w, t_h)

1920 2880.0


In [4]:
def get_image_parameters(ip_img):
    
    image_arr=numpy.array(Image.open(ip_img),dtype=numpy.uint8)
    img_h, img_w, layers = image_arr.shape
    
    return img_h, img_w

In [5]:
def get_horizontal_panning_parameters(img_w, total_frames, op_resolution):
    
    desired_w, desired_h = op_resolution
    pixels_to_move_per_frame = (img_w-desired_w)/float(total_frames) 
    
    return pixels_to_move_per_frame

In [42]:
def get_horizontal_image_panning_parameters(img_h, img_w, frame_count, total_frames, op_resolution, pixels_to_move_per_frame):
    
    desired_w, desired_h = op_resolution

    # Width start and end will depend on frames to be moved 
    x_start = round((frame_count-1)*pixels_to_move_per_frame)
    x_width = desired_w
    
    # height would remain the same
    y_start = 0
    y_height = img_h
    
    return int(x_start), int(x_width), int(y_start), int(y_height)

In [7]:
def crop_image(ip_img, x_start, x_width, y_start, y_height):
    
    image_arr=numpy.array(Image.open(ip_img),dtype=numpy.uint8)
    
    crop_arr=image_arr[y_start:y_start+y_height, x_start:x_start+x_width, :]
    
    crop_image = Image.fromarray(crop_arr,mode="RGB")
    
    return crop_image

## Test function for cropping an image

In [16]:
INPUT_DIR = '/Users/talat/Pictures/_Photos/longexp/'
sub_dir = '2020-05-14 - YEG Balcony Turtle Garden (TL) E'

ip_dir = os.path.join(INPUT_DIR, sub_dir)
ip_file = os.path.join(ip_dir, 'IMG_3816.jpg')

res_hd = (1920, 1080)
res_4k = (3840, 2160)
res = res_hd

frame_rate = 24

x_width, y_height = res

x_start = 0
y_start = 0


crop_img = crop_image(ip_file, x_start, x_width, y_height, x_width)
crop_img.save("Average.png")
crop_img.show()

# cv2.imshow("cropped", crop_img)

#crop_img.save('temp.jpg', format = 'JPEG', quality = 100,
#                     icc_profile = Image.open(ip_file).info.get('icc_profile',''))

## Test function to check panning for a single input directory

In [66]:
INPUT_DIR = '/Users/talat/Pictures/_Photos/longexp/'
sub_dir = '2020-05-14 - YEG Balcony Turtle Garden (TL) E'

ip_dir = os.path.join(INPUT_DIR, sub_dir)
OUTPUT_DIR = '/Users/talat/Pictures/_Photos/Video/'

res_hd = (1920, 1080)
res_4k = (3840, 2160)
res_hor_panning_portrait = (3000, 0)
res = res_hor_panning_portrait

frame_rate = 24
ext = '.jpg'
panning_style = 'h'
aspect_ratio=(2, 3)
vid_ext = '.mp4'

op_dir = OUTPUT_DIR

create_video_panning (ip_dir, op_dir, res, frame_rate, ext, panning_style, aspect_ratio, vid_ext)

4000 6000 8.333333333333334 240 (3000, 4000)
Output Video:  /Users/talat/Pictures/_Photos/Video/2020-05-14 - YEG Balcony Turtle Garden (TL) E_3000x4000.mp4 



## Test function to apply panning for all subdirectiories within a directory

In [68]:
INPUT_DIR = '/Users/talat/Pictures/_Photos/longexp/'
OUTPUT_DIR = '/Users/talat/Pictures/_Photos/Video/'

# resolution = (width, height)
res_hd = (1920, 1080)
res_4k = (3840, 2160)
res_hor_panning_portrait = (3000, 0)
res = res_hor_panning_portrait

frame_rate = 24
ext = '.jpg'
panning_style = 'h'
aspect_ratio=(2, 3)
vid_ext = '.mp4'

sub_dir_list = [f for f in os.listdir(INPUT_DIR) if os.path.isdir(os.path.join(INPUT_DIR,f))]
sub_dir_list.sort()

for sub_dir in sub_dir_list:
    
    ip_dir = os.path.join(INPUT_DIR, sub_dir)
    op_dir = OUTPUT_DIR
    
    print(sub_dir, end=' : ')
    
    create_video_panning (ip_dir, op_dir, res, frame_rate, ext, panning_style, aspect_ratio, vid_ext)
    # create_video(ip_dir, op_dir, res, frame_rate, ext)

2020-05-14 - YEG Balcony Turtle Garden (TL) E : 4000 6000 8.333333333333334 240 (3000, 4000)
Output Video:  /Users/talat/Pictures/_Photos/Video/2020-05-14 - YEG Balcony Turtle Garden (TL) E_3000x4000.mp4 

2020-05-14 - YEG Balcony Turtle Garden (TL) E copy : 4000 6000 3.3003300330033003 606 (3000, 4000)
Output Video:  /Users/talat/Pictures/_Photos/Video/2020-05-14 - YEG Balcony Turtle Garden (TL) E copy_3000x4000.mp4 

